# 02 Pytorch Network
#### ＝＝＝ 目次 ＝＝＝
0. ライブラリの呼び出し
1. Layer
2. モデルの定義
3. モデルの学習
4. モデルの保存・読み込み

補足資料

5. 学習済みモデル
6. 自作loss関数

---
## 0. ライブラリの呼び出し
- [torch.nn](https://pytorch.org/docs/stable/nn.html)：NNを構成するためのモジュール (パラメータあり，クラス)
- [torch.nn.functional](https://pytorch.org/docs/stable/nn.functional.html)：様々な関数を内包するモジュール (パラメータなし，関数)
- [torch.optim](https://pytorch.org/docs/stable/optim.html)：optimizerを内包するモジュール
- torchsummary*：NNの構造を把握するためのモジュール

*pytorchライブラリに含まれていないため`pip install torch-summary`でインストールが必要

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchsummary import summary

---
## 1. Layer

|<div align='center'>layer</div>|<div align='center'>program</div>|<div align='center'>例</div>|
|---|---|---|
|<div align='left'>全結合層</div>|<div align='left'>nn.Linear(in_features, out_features)</div>|<div align='left'>nn.Linear(64, 10)</div>|
|<div align='left'>畳み込み層</div>|<div align='left'>nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)</div>|<div align='left'>nn.Conv2d(3, 64, 3, 1, 1)</div>|
|<div align='left'>プーリング層</div>|<div align='left'>nn.MaxPool2d(kernel_size, stride)</div>|<div align='left'>nn.MaxPool2d(2, 2)</div>|
|<div align='left'>LSTM層</div>|<div align='left'>nn.LSTM(input_size, hidden_size, num_layers, bias, batch_first, dropout, bidirectional)</div>|<div align='left'>nn.MaxPool2d(2, 2)</div>|
|<div align='left'>バッチ正規化</div>|<div align='left'>nn.BatchNorm2d(num_features)</div>|<div align='left'>nn.BatchNorm2d(64)</div>|
|<div align='left'>ドロップアウト</div>|<div align='left'>nn.Dropout(p)</div>|<div align='left'>nn.Dropout(0.5)</div>|

|<div align='center'>活性化関数</div>|<div align='center'>program (nn)</div>|<div align='center'>program (F)</div>|
|---|---|---|
|<div align='left'>ReLU</div>|<div align='left'>nn.ReLU(inplace=True)</div>|<div align='left'>F.relu</div>|
|<div align='left'>Sigmoid</div>|<div align='left'>nn.Sigmoid()</div>|<div align='left'>F.sigmoid</div>|
|<div align='left'>Tanh</div>|<div align='left'>nn.Tanh()</div>|<div align='left'>F.tanh</div>|
|<div align='left'>Softmax</div>|<div align='left'>nn.Softmax(dim=1)</div>|<div align='left'>F.softmax</div>|

|<div align='center'>Loss関数</div>|<div align='center'>program</div>|
|---|---|
|<div align='left'>Mean Squared Error</div>|<div align='left'>nn.MSELoss()</div>|
|<div align='left'>Mean Average Error</div>|<div align='left'>nn.L1Loss()</div>|
|<div align='left'>Cross Entropy$^{*1}$</div>|<div align='left'>nn.CrossEntropyLoss()</div>|
|<div align='left'>Binary Cross Entropy</div>|<div align='left'>nn.BCELoss()</div>|
|<div align='left'>Binary Cross Entropy with Sigmoid</div>|<div align='left'>nn.BCEWithLogitsLoss()</div>|

$^{*1}$ softmax関数を内包していることに注意

---
#### 全結合層
10次元 → 8次元

In [ ]:
# 100個の10次元データ
batch_size = 100
dim_input = 10
inputs = torch.randn(batch_size, dim_input)
print(inputs.shape)

In [ ]:
# 全結合層の定義
fc = nn.Linear(in_features=10, out_features=8)
print(fc)
print("weight :", fc.weight.shape)

In [ ]:
outputs = fc(inputs)

print("input :", inputs.shape)
print("output :", outputs.shape)

---
#### 畳み込み層
- 入力チャンネル：3
- 出力チャンネル：64
- カーネルサイズ：3
- ストライド：1
- ゼロパディング

入力データ：10×10の32チャンネル画像
$$(N, C, H, W) = (N, 32, 10, 10)^{*2}$$

$^{*2}$ PytorchはChannel Firstの形式

In [ ]:
# 100個の10×10の32チャンネル画像
batch_size = 100
inputs = torch.randn(batch_size, 32, 10, 10)
print(inputs.shape)

In [ ]:
conv = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
print(conv)
print("weight :", conv.weight.shape)

In [ ]:
outputs = conv(inputs)

print("input :", inputs.shape)
print("output :", outputs.shape)

---
#### プーリング層
- カーネルサイズ：2
- ストライド：2

In [ ]:
# 100個の10×10の32チャンネル画像
batch_size = 100
inputs = torch.randn(batch_size, 32, 10, 10)
print(inputs.shape)

In [ ]:
pool = nn.MaxPool2d(2, 2)
print(pool)

In [ ]:
outputs = pool(inputs)

print("input :", inputs.shape)
print("output :", outputs.shape)

#### Global Max Pooling
pytorchにはGlobal Max Poolingが用意されていないので，`nn.AdaptiveMaxPool2d`を用いる

In [ ]:
# 100個の10×10の32チャンネル画像
batch_size = 100
inputs = torch.randn(batch_size, 32, 10, 10)

In [ ]:
gmp = nn.AdaptiveMaxPool2d((1, 1))
print(gmp)

In [ ]:
outputs = gmp(inputs)

print("input :", inputs.shape)
print("output :", outputs.shape)

In [ ]:
outputs = outputs.squeeze()
print("output :", outputs.shape)

---
#### LSTM層
- 入力サイズ：20
- 出力サイズ：10

入力データ：(バッチサイズ, 最大シーケンス長, 特徴量次元数) = (100, 50, 20)

In [ ]:
# 100個の10×10の32チャンネル画像
batch_size = 100
inputs = torch.randn(batch_size, 50, 20)

In [ ]:
lstm = nn.LSTM(input_size=20, hidden_size=10, batch_first=True)
print(lstm)

In [ ]:
outputs, (hiddens, cells) = lstm(inputs)
print(outputs.shape)
print(hiddens.shape)
print(cells.shape)

---
#### 活性化関数

In [ ]:
# 100個の10×10の32チャンネル画像
batch_size = 100
inputs = torch.randn(batch_size, 32, 10, 10)
print(inputs.shape)

In [ ]:
# nn.ReLU
relu = nn.ReLU(inplace=True)
print(relu)

outputs = relu(inputs)
print(outputs.shape)

In [ ]:
# F.relu
outputs = F.relu(inputs)
print(outputs.shape)

---
#### 損失関数(loss)

pytorchでは，loss関数の変数名は慣例で`criterion`が多い

loss関数を通した実数値の変数名は`loss`が多い

In [ ]:
targets = torch.ones_like(outputs)
criterion = nn.MSELoss()
loss = criterion(outputs, targets)
print(loss)

---
## 2. モデルの定義
NNは`nn.Module`を継承したクラスとして定義する．

`__init__`と`forward`メソッドを必ず記述する．
- `__init__`：パラメータを持つ層(nn)を定義する
- `forward`：モデルの入力から出力までの計算を記述する(initで定義した層+Fで記述)

シンプルな実装

In [ ]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(in_features=64 * 7 * 7, out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=84)
        self.fc3 = nn.Linear(in_features=84, out_features=10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool(x)
        
        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool(x)
        
        x = x.view(-1, 64 * 7 * 7)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
        return x

`summary(model, input)`：モデル構造や特徴マップのサイズ確認

In [ ]:
model = Net()
inputs = torch.randn(100, 1, 28, 28)
summary(model, inputs)
print(model)

#### Sequentialを用いた実装
`nn.Sequential`を用いることで，各層をまとめて定義することができる．

In [ ]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.conv = nn.Sequential(nn.Conv2d(1, 32, 3, padding=1), nn.ReLU(inplace=True), nn.MaxPool2d(2, 2),
                                  nn.Conv2d(32, 64, 3, padding=1), nn.ReLU(inplace=True), nn.MaxPool2d(2, 2))
        
        self.fc = nn.Sequential(nn.Linear(64 * 7 * 7, 120), nn.ReLU(inplace=True),
                                nn.Linear(120, 84), nn.ReLU(inplace=True),
                                nn.Linear(84, 10))

    def forward(self, x):
        x = self.conv(x)
        x = self.fc(x.view(-1, 64 * 7 * 7))
        return x

In [ ]:
model = Net()
summary(model, torch.randn(100, 1, 28, 28))
print(model)

#### モデルの計算（順伝播）
例なので乱数を入力データとする

In [ ]:
# summaryを使用するとmodelはGPUに送られる
model = model.to("cpu")

# 100個の28×28の1チャンネル画像
batch_size = 100
inputs = torch.randn(batch_size, 1, 28, 28)
print(inputs.shape)

In [ ]:
# 順伝播
outputs = model(inputs)
print(outputs.shape)

#### モデルの各層の取得

In [ ]:
model.conv

In [ ]:
model.conv[0]

#### モデルのパラメータ
`model.parameters()`でモデルのパラメータをgeneratorとして取得できる

optimizerの定義で渡す

In [ ]:
model.parameters()

#### モデルのphase

In [ ]:
# モデルを訓練モードに
model.train()

In [ ]:
# モデルを検証モードに
model.eval()

---
## 3. モデルの学習
1. 学習データを用意
2. モデルの定義
3. loss関数，optimizerを定義
4. データをモデルに入力し出力，lossを計算
5. backpropagation，パラメータ更新

#### 3-1. 学習データを用意
今回は人工データ(乱数)を用いる
- データ数(バッチサイズ)：100
- 入力データ：28×28の1チャンネル画像
- 教師データ：10クラスのラベル（pytorchではone-hot化しない）

In [ ]:
batch_size = 100

In [ ]:
# 100個の28×28の1チャンネル画像
inputs = torch.randn(batch_size, 1, 28, 28)
print(inputs.shape)

In [ ]:
# 100個の10クラスの教師データ
targets = torch.empty(batch_size, dtype=torch.long).random_(10)
print(targets.shape)
print(targets)

GPUが使える場合は，読み込んだデータをdeviceに送る

In [ ]:
# GPUが使えるかを確認
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("使用デバイス：", device)

# GPUが使えるならGPUにデータを送る
inputs = inputs.to(device)
targets = targets.to(device)

---
#### 3-2. モデルの定義
2.で定義したモデルを用いる

In [ ]:
# モデルの定義
model = Net()

In [ ]:
# マルチGPUが使える場合
if torch.cuda.device_count() > 1:
    print("Let's use", torch.cuda.device_count(), "GPUs")
    model = nn.DataParallel(model)

model.to(device) # モデルをGPUへ

---
#### 3-3. loss関数，optimizerを定義
- loss関数：クロスエントロピー
- optimizer：SGD

In [ ]:
# loss関数の定義
criterion = nn.CrossEntropyLoss()

In [ ]:
# optimizerを定義
optimizer = optim.SGD(model.parameters(), lr=0.1)

---
#### 3-4. データをモデルに入力し出力，lossを計算

In [ ]:
# モデルを訓練モードに
model.train()

# 勾配を初期化
optimizer.zero_grad() 

In [ ]:
# 順伝播
outputs = model(inputs)
loss = criterion(outputs, targets)
print(loss)

---
#### 3-5. backpropagation，パラメータ更新

In [ ]:
loss.backward()  # backpropagation
optimizer.step() # 勾配を元にパラメータを更新

これを，epoch，batch，train or validationでそれぞれfor文で回せばミニバッチ学習となる．

---
## 4. モデルの保存・読み込み
|<div align='center'>Save</div>|<div align='center'>意味</div>|
|---|---|
|<div align='left'>model.state_dict()</div>|<div align='left'>モデルの学習可能なパラメータを返す</div>|
|<div align='left'>torch.save(state_dict$^{*3}$, model_path$^{*4}$)</div>|<div align='left'>state_dictを指定したpathに保存する</div>|

|<div align='center'>Load</div>|<div align='center'>意味</div>|
|---|---|
|<div align='left'>torch.load(model_path)</div>|<div align='left'>指定したpathのstate_dictを読み込む</div>|
|<div align='left'>model.load_state_dict(state_dict)</div>|<div align='left'>state_dictをモデルに復元する</div>|

$^{*3}$ モデルをGPUに送っている場合はCPUに戻してからstate_dictを取得する (エラー回避のため)

$^{*4}$ 保存するファイルの拡張子は`.pth`が慣例

In [ ]:
#保存用ディレクトリを作成
import os
os.makedirs("result", exist_ok=True)

# モデルを保存する。
torch.save(model.to('cpu').state_dict(), "result/model.pth")

In [ ]:
# 保存したモデルを読み込む。
model = Net()
model.load_state_dict(torch.load("result/model.pth"))
print(model)

---
# 補足資料
## 5. 学習済みモデル
[torchvision](https://pytorch.org/docs/stable/torchvision/index.html)：コンピュータビジョンにおける有名なデータセット(MNIST，COCOなど)，モデルアーキテクチャ，画像変換処理から構成される

In [ ]:
import torchvision

In [ ]:
# pretrained=Trueにすることで，学習済みのパラメータを設定
vgg = torchvision.models.vgg16(pretrained=True)

In [ ]:
print(vgg)

---
## 6. 自作loss関数
モデルの定義と同様に`nn.Module`を継承したクラスとして定義する

In [ ]:
class MyLoss(nn.Module):

    def __init__(self):
        super(MyLoss, self).__init__()

    def forward(self, outputs, targets):
        return torch.mean(torch.pow((outputs - targets),2))

In [ ]:
batch_size = 100
outputs =  torch.randn(batch_size, 3, 28, 28)
targets =  torch.randn(batch_size, 3, 28, 28)

In [ ]:
criterion = MyLoss()
loss = criterion(outputs, targets)
print(loss)